<a href="https://colab.research.google.com/github/Dinarque/INALCO_Inalco_M2_langage_de_scripts_2024_eleves/blob/main/algorithmique/interview_controle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interview controle 2

Traitez le problème suivant. Indiquez et justifiez la complexité de votre solution


On vous donne un tableau d'entiers 0-indexé nums de longueur n. Vous êtes initialement positionné à nums[0].

Chaque élément nums[i] représente la longueur maximale d'un saut en avant depuis l'index i. En d'autres termes, si vous êtes à nums[i], vous pouvez sauter vers n'importe quel nums[i + j] où : 0≤j≤nums[i] et i+j<n

Retournez le nombre minimal de sauts pour atteindre nums[n - 1]



Exemple 1 :

Entrée : nums = [2, 3, 1, 1, 4]

Sortie : 2

Explication : Le nombre minimal de sauts pour atteindre le dernier index est 2. Sauter 1 pas de l'index 0 à l'index 1, puis 3 pas jusqu'à l'index final.

In [5]:
ex =  [2, 3, 1, 1, 4]

# Solution 1 : Approche récursive


La différence par rapport à une fonction récursive  pour l interview précédente est qu on ne veut pas forcément commencer du début, on "commence par l'inverse" en voyant combien de sauts il faut depuis l index de départ pour arriver à la fin

la clé du pb est la condition de mise à jour.
a forumuler en langage naturel d abord

In [10]:
def minimum_jumps(nums, index_de_depart=0):


    # cas trivial
    if index_de_depart >= len(nums) - 1:
        return 0

    # Si nums[index_de_depart] est 0, on ne peut pas avancer
    if nums[index_de_depart] == 0:
        return float('inf')

    # Explorer tous les sauts possibles depuis cet index
    min_steps = float('inf')
    for step in range(1, nums[index_de_depart] + 1):
        min_steps = min(min_steps, 1 + minimum_jumps(nums, index_de_depart + step))
        # on teste le minimum de steps pour atteindre la fin depuis chaque etape de l intervalle atteignable
        # on peut avoir cette info de façon récursive, et remonter de la fin ou c est trivial ou début

    return min_steps


In [11]:
minimum_jumps(ex)

2

Complexité exponentielle , tres mauvais
on peut améliorer en stockant de façon externe les cases déjà visitées,
réduit à O(n2)




# Solution 2 : stocker les valeurs



approche dite "programmation dynamique"

In [14]:
def minimum_jumps(nums):

    if len(nums) <= 1:
        return 0  # Aucun saut nécessaire si le tableau a une seule case ou est vide.

    # On crée un tableau stockant le nombre de saut minimum pour aller à la fin et on va le mettre à jour au fur et à mesure de l'algo
    min_to_reach_i = [float('inf')] * len(nums)

    min_to_reach_i[0] = 0  # Il faut 0 saut pour arriver à la première position.

    for i in range(len(nums)):
        for j in range(1, nums[i] + 1):
            if i + j < len(nums):
                min_to_reach_i[i + j] = min(min_to_reach_i[i + j], min_to_reach_i[i] + 1)

    return min_to_reach_i[-1]

In [15]:
minimum_jumps(ex)

2

O(n2) ,
 le cas de base de la programmation dynamique ou la solution au rang n dépend de celle des rangs précédents.  ici on fait l update de façon progressive quitte à dépasser i dans min_to_reach, bien pus efficace que je juste se préoccuper de i

# Solution 3 : rapide et efficace

On essaie de voir à la case i le plus loin où on peut aller

L'idée consiste à gérer au mieux les variables

On a une variable j qui stocke le nombre de sauts minimum effectué pour atteindre la case i

L'idée est de distinguer la variable représentant la case la plus lointaine qu'on peut atteindre en tout dans le jeu est celle la plus loin qu'on peut atteindre en j sauts

C'est seulement quand on dépasse

In [17]:
def minimum_jumps(nums):
        """
        :type nums: List[int]
        :rtype: int
        """

        # Cas trivial
        if len(nums) <= 1:
            return 0

        jumps = 0
        current_end = 0 # intervalle atteignable avec jumps sauts , "intervalle d'atteignabilité avec le nombre de sauts déjà réalisés "
        farthest = 0

        for i in range(len(nums)):
            # Mettez à jour le farthest (l'indice le plus loin qu'on peut atteindre)
            # Le nouveau max atteignable est soit l'ancien soit le plus grand saut qu'on puisse faire depuis la case i
            farthest = max(farthest, i + nums[i])

            # Si on atteint la fin de l'intervalle actuel, on fait un saut pour pouvoir aller plus loin
            if i == current_end:
                jumps += 1
                current_end = farthest #l'intervalle atteignable avec un saut supplémentaire devient le maximum qu'on peut atteindre
                # Si le farthest atteint ou dépasse la fin du tableau, on peut arrêter
                if current_end >= len(nums) - 1:
                    break

        return jumps

In [18]:
minimum_jumps(ex)

2

# Bonus : recherche en largeur

Considérez chaque index comme un noeud dans un graphe, avec les sauts possibles comme arêtes. Utilisez une recherche en largeur pour trouver le chemin le plus court vers le dernier noeud.
s inspire de la théorie des automates

In [20]:
from collections import deque

def minimum_jumps(nums):
    n = len(nums)
    if n <= 1:
        return 0

    queue = deque([(0, 0)])  # (index atteint,  nombre de sauts nécessaires pour atteindre )
    # Pour chaque nombre de sauts on regarde tout ce qu'on peut atteindre
    # puis on regarde à partir de tous ces noeuds atteignables ceux qu'on peut atteindre avec un saut de plus
    # on ajoute ces découvertes à la queue
    # On arrête quand on atteint la fin.
    # ça marche car on est dans un DAG (pas de cycles, même si on aurait pu s'en tirer avec un filtre sur l indice)
    visited = set()

    while queue:
        print(queue)
        index, jumps = queue.popleft()
        if index == n - 1:
            return jumps

        for step in range(1, nums[index] + 1):
            next_index = index + step
            if next_index < n and next_index not in visited:
                visited.add(next_index)
                queue.append((next_index, jumps + 1))

    return float('inf')

In [21]:
minimum_jumps(ex)

deque([(0, 0)])
deque([(1, 1), (2, 1)])
deque([(2, 1), (3, 2), (4, 2)])
deque([(3, 2), (4, 2)])
deque([(4, 2)])


2

# Programmation Objet : dictionnaires expérimentaux

# A) le dictionnaire à clé

On veut sécuriser le dictionnaire ! Seules les personnes qui ont la clé secrète doivent pouvoir lire les valeurs.

Implémentez une classe de dictionnaire telle que :
* le dictionnaire peut être initialisé optionellement avec un password(str). Si il n’en a pas il se comportera comme un dictionnaire normal.
* Sinon quand on veut modifier la valeur d’une clé il faudra donner le mot de passe pour éditer une valeur
dic[“a”, password] = 3
* si le mot de passe est faux une erreur s’affiche


In [34]:
class DictionnaireACle:
    def __init__(self, password=None):
        self.data = {}
        self.password = password  # besoin d'un nouveau champs dans l'init

    def __getitem__(self, key):
        # rien ne change, il faut juste recoder en utilisant les données
        return self.data[key]

    def __setitem__(self, key, value):
        # Ici la clé du problème est de considérer que la clé est un tuple et plus une valeur simple pour avoir le fonctionnement attendu

        if self.password:
            if not isinstance(key, tuple) or len(key) != 2:
                raise ValueError("Ce dictionnaire est protégé. Utilisez un tuple (clé, mot de passe) comme clé.")
            actual_key, password = key
            if password != self.password:
                raise ValueError("Mot de passe incorrect.")
            self.data[actual_key] = value
        else:
            self.data[key] = value


In [33]:
dic1 = DictionnaireACle()
dic1["a"] = 1
print(dic1["a"])  # Affiche 1

dic2 = DictionnaireACle(password="secret123")
dic2["a", "secret123"] = 42
dic2["b", "secret123"] = 99

# Lecture des valeurs
print(dic2["a"])  # Affiche : 42
print(dic2["b"])  # Affiche : 99

# Modification d'une valeur avec mot de passe
dic2["a", "secret123"] = 100
print(dic2["a"])  # Affiche : 100

# Essai d'accès sans mot de passe (Erreur)
try:
    dic2["c"] = 5
except ValueError as e:
    print(e)

# Essai avec un mot de passe incorrect (Erreur)
try:
    dic2["b", "wrongpassword"] = 123
except ValueError as e:
    print(e)

1
42
99
100
Ce dictionnaire est protégé. Utilisez un tuple (clé, mot de passe).
Mot de passe incorrect.


# Autre option : tirer parti de l'héritage en faisant hériter de dict

mieux car du coup votre dictionnaire à clé n'est pas juste un objet qui contient un dict data, c'est vraiment un dictionnaire !!

In [36]:
class DictionnaireACleHeritage(dict):
    def __init__(self, password=None, *args, **kwargs):

        super().__init__(*args, **kwargs)
        self._password = password

    def __getitem__(self, key):  # Pas nécessaire grace à l'héritage
        return super().__getitem__(key)

    def __setitem__(self, key, value):
        if self._password:
            if isinstance(key, tuple) and len(key) == 2:
                key, password = key
                if password == self._password:
                    super().__setitem__(key, value)
                else:
                    raise ValueError("Mot de passe incorrect.")
            else:
                raise ValueError(
                    "Ce dictionnaire est protégé. Utilisez un tuple (clé, mot de passe)."
                )
        else:
            super().__setitem__(key, value)

In [38]:
dic1 = DictionnaireACleHeritage()
dic1["a"] = 1
print(dic1["a"])  # Affiche 1

dic2 = DictionnaireACleHeritage(password="secret123")
dic2["a", "secret123"] = 42
dic2["b", "secret123"] = 99

# Lecture des valeurs
print(dic2["a"])  # Affiche : 42
print(dic2["b"])  # Affiche : 99

# Modification d'une valeur avec mot de passe
dic2["a", "secret123"] = 100
print(dic2["a"])  # Affiche : 100

# Essai d'accès sans mot de passe (Erreur)
try:
    dic2["c"] = 5
except ValueError as e:
    print(e)

# Essai avec un mot de passe incorrect (Erreur)
try:
    dic2["b", "wrongpassword"] = 123
except ValueError as e:
    print(e)

1
42
99
100
Ce dictionnaire est protégé. Utilisez un tuple (clé, mot de passe).
Mot de passe incorrect.


# B) Le dictionnaire stockeur

Normalement quand on attribue une valeur à un dictionnaire, comme dic[“a”] = 3, une réaffectation comme dic[“a”] = 2 vient écraser l’ancienne valeur.
Mais on voudrait pouvoir les conserver et faire en sorte que le dictionnaire ait une mémoire.
Créez une classe de dictionnaires tel que :
* la valeur renvoyée par l’appel dic[k] soit la dernière valeur affectée
* lors d’une nouvelle affectation l’ancienne valeur n’est pas détruite mais le dictionnaire stocke une liste des anciennes valeurs affectées.
* le dictionnaire a une méthode dic.history(a) qui renvoie l’historique des valeurs
* le dictionnaire a une méthode dic.rewind(a) qui supprime la dernière affectation d’une valeur à la clé a si possible
Ex :
dic[“a”] = 2
dic[“a”] = 3
print(dic[“a”]) => 3
print(dic.history(a)) => [2,3]
dic.rewind(a)
print(dic[“a”]) => 2




In [28]:
class DictionnaireStockeur:

    def __init__(self):
      # l'idée est qu'on redéfinit les données comme étant une liste de valeurs et plus juste une seule valeur
        self.data = {}

    def __getitem__(self, key):
        return self.data[key][-1] if key in self.data else None # Cela impacte la recherche de la valeur (retourner le dernier élément de la liste)

    def __setitem__(self, key, value):  # et l affectation qui est un ajout à une liste
        if key not in self.data:
            self.data[key] = []
        self.data[key].append(value)

    def history(self, key):
        # Retourne l'historique des valeurs pour une clé donnée, là il suffit de retourner toute la liste
        if key in self.data:
            return self.data[key]
        else:
            return []

    def rewind(self, key):
        # Supprime la dernière valeur si possible
        if key in self.data and len(self.data[key]) > 1:
            self.data[key].pop()
        elif key in self.data and len(self.data[key]) == 1:
            del self.data[key]

In [29]:
dic = DictionnaireStockeur()

dic["a"] = 2
dic["a"] = 3
print(dic["a"])
print(dic.history("a"))
dic.rewind("a")
print(dic["a"])

3
[2, 3]
2
